### **1.Definir o Escopo do Chatbot**

Determine as principais funcionalidades e objetivos do chatbot. Ele fornecerá recomendações de filmes, informações sobre filmes específicos, ou realizará outras tarefas relacionadas a filmes?

Nosso chatbot vai ser capaz de identificar as seguintes inteções e responder elas: <br>
    1. Assistir filme <br>
    2. Comprar ingresso <br>
    3. Querer saber detalhes de um filme específico <br>
    4. Checar as sessões de um filme específico <br>

### **2.Coletânea de dados**
Coletar e entender os datasets reunidos

In [2]:
import pandas as pd
# origem do dataset : https://github.com/google-research-datasets/ccpe/tree/main
df_conversations = pd.read_json('datasets\data.json')

#### Anotações imporatantes de cada label desse dataset<br>
<br>
ENTITY_NAME: These mark the names of relevant entities mentioned. <br>
ENTITY_PREFERENCE: These are defined as statements indicating that the dialog participant does or does not like the relevant entity in general, or that they do or do not like some aspect of the entity. This may also be thought of the participant having some sentiment about what is being discussed. <br>
ENTITY_DESCRIPTION: Neutral descriptions that describe an entity but do not convey an explicit liking or disliking. <br>
ENTITY_OTHER: Other relevant statements about an entity that convey relevant information of how the participant relates to the entity but do not provide a sentiment. Most often, these relate to whether a participant has seen a particular movie, or knows a lot about a given entity. <br>

In [3]:
print("Número de conversas:", len(df_conversations))
print("\nPrimeira conversa:")
print(df_conversations.iloc[0])


conversation_id = df_conversations.iloc[0]['conversationId']
print("\nID da primeira conversa:", conversation_id)

utterances = df_conversations.iloc[0]['utterances']
print("\nNúmero de utterances na primeira conversa:", len(utterances))

# Acessando a primeira utterance
text_utterance_1 = utterances[0]['text']
print("\nTexto da primeira utterance:", text_utterance_1)

# Segunda instância
text_utterance_2 = utterances[1]['text']
segments_utterance_2 = utterances[1]['segments']

print("\nTexto da segunda utterance:", text_utterance_2)
print("Entidades anotadas na segunda utterance:")
for segment in segments_utterance_2:
    annotated_text = segment['text']
    entities = segment['annotations']
    print(f"Texto: {annotated_text}, Entidades: {entities}")

Número de conversas: 502

Primeira conversa:
conversationId                                           CCPE-8e113
utterances        [{'index': 0, 'speaker': 'ASSISTANT', 'text': ...
Name: 0, dtype: object

ID da primeira conversa: CCPE-8e113

Número de utterances na primeira conversa: 43

Texto da primeira utterance: generally speaking what type of movies do you watch

Texto da segunda utterance: I like thrillers a lot.
Entidades anotadas na segunda utterance:
Texto: thrillers, Entidades: [{'annotationType': 'ENTITY_NAME', 'entityType': 'MOVIE_GENRE_OR_CATEGORY'}]
Texto: I like thrillers a lot, Entidades: [{'annotationType': 'ENTITY_PREFERENCE', 'entityType': 'MOVIE_GENRE_OR_CATEGORY'}]


In [5]:
#dataset com informações dos filmes
df_info = pd.read_csv('datasets/IMDB.csv')
df_info.head()

,Unnamed: 0,index,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,...,averageRating,numVotes,ordering,title,region,language,types,attributes,isOriginalTitle,Description
0,0,0,tt0102926,movie,The Silence of the Lambs,The Silence of the Lambs,0,1991,\N,118,...,8.6,1473918,50,The Silence of the Lambs,US,en,\N,\N,0,"Jodie Foster stars as Clarice Starling, a top ..."
1,1,1,tt0103064,movie,Terminator 2: Judgment Day,Terminator 2: Judgment Day,0,1991,\N,137,...,8.6,1128166,17,Terminator 2: Judgment Day,US,en,dvd,\N,0,"In this sequel set eleven years after ""The Ter..."
2,2,3,tt0110357,movie,The Lion King,The Lion King,0,1994,\N,88,...,8.5,1090882,18,The Lion King 3D,US,en,\N,3-D version,0,This Disney animated feature follows the adven...
3,3,4,tt0110912,movie,Pulp Fiction,Pulp Fiction,0,1994,\N,154,...,8.9,2118762,22,Pulp Fiction,US,en,\N,\N,0,Vincent Vega (John Travolta) and Jules Winnfie...
4,4,5,tt0111161,movie,The Shawshank Redemption,The Shawshank Redemption,0,1994,\N,142,...,9.3,2759621,2,The Shawshank Redemption,US,en,\N,\N,0,Andy Dufresne (Tim Robbins) is sentenced to tw...


### **3.Pré-processamento dos dados**
Realizar a limpeza, pré-processamento. Tokenização, remoção de stop words, etc.

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Carregar o dataset
dataset_path = 'datasets/conversas_filmes.csv'
df = pd.read_csv(dataset_path)

# Dividir o dataset em treino e teste
train_data, test_data, train_labels, test_labels = train_test_split(
    df['Conversa'], df['Intenção'], test_size=0.2, random_state=42
)

# Criar um vetorizador para converter as conversas em representação numérica
vectorizer = CountVectorizer()
train_vectors = vectorizer.fit_transform(train_data)
test_vectors = vectorizer.transform(test_data)

### **4.Identifique Intenções e Entidades**
Defina as intenções que seu chatbot precisa entender (por exemplo, "Encontrar um filme" ou "Obter informações sobre um filme específico") e as entidades associadas (por exemplo, título do filme, gênero, diretor).

In [17]:
# Criar e treinar o modelo Naive Bayes
classifier = MultinomialNB()
classifier.fit(train_vectors, train_labels)

# Fazer previsões no conjunto de teste
predictions = classifier.predict(test_vectors)

# Avaliar o desempenho do modelo
accuracy = accuracy_score(test_labels, predictions)
print(f'Acurácia: {accuracy:.2f}\n')

classification_rep = classification_report(test_labels, predictions)
print('Relatório de Classificação:\n', classification_rep)

# Exemplo de uso do modelo
new_conversation = ["Qual é o filme mais recente em cartaz?"]
new_conversation_vector = vectorizer.transform(new_conversation)
prediction = classifier.predict(new_conversation_vector)
print(f'\nPrevisão para a nova conversa: {prediction[0]}')

Acurácia: 0.94

Relatório de Classificação:
                    precision    recall  f1-score   support

   Assistir filme       1.00      0.96      0.98        24
   Checar sessões       0.81      1.00      0.90        22
 Comprar ingresso       1.00      0.90      0.95        21
Detalhes do filme       1.00      0.88      0.93        16

         accuracy                           0.94        83
        macro avg       0.95      0.93      0.94        83
     weighted avg       0.95      0.94      0.94        83


Previsão para a nova conversa: Assistir filme


### **5.Crie um Modelo de Linguagem**
Treine um modelo de linguagem para entender e gerar respostas relevantes.

### **6.Construa uma Base de Conhecimento**
Compile uma base de dados ou fonte de conhecimento que seu chatbot possa acessar para fornecer informações sobre filmes. Isso pode incluir bancos de dados de filmes, APIs de informações sobre filmes, etc.

### **7.Desenvolva a Lógica de Conversação**
Crie fluxos de conversação para lidar com diferentes cenários. Isso envolve definir respostas apropriadas para diferentes intenções e entidades identificadas.

### **8.Integre Serviços de Terceiros**
Integre serviços externos, como APIs de informações sobre filmes, para enriquecer as respostas do seu chatbot.

### **9.Teste o Chatbot**
Realize testes extensivos para garantir que o chatbot compreenda corretamente as consultas dos usuários e forneça respostas precisas.


### **10.Implemente e Monitore**
Implemente o chatbot na plataforma escolhida e monitore seu desempenho. Esteja preparado para fazer ajustes conforme necessário.